# SQL 작성

In [1]:
import sys
import os
import pandas as pd
from scipy import io
import xml.etree.ElementTree as ET
import cv2
from glob import glob
sys.path.append('../../src')
import DISData as DD

## SQL 연결

In [11]:
doUT = DD.SQL()

host:  192.168.0.128
user:  diadmin
password:  ········


SQL connection done


## query 작성

In [3]:
query1 = (f'''CREATE TABLE IF NOT EXISTS zer01ne_box_info(
            id INT PRIMARY KEY AUTO_INCREMENT,
            img_name VARCHAR(255) NOT NULL,
            img_dir VARCHAR(1000) NOT NULL,
            img_format VARCHAR(10) NOT NULL,
            img_width INT NOT NULL,
            img_height INT NOT NULL,
            color_space VARCHAR(10) NOT NULL,
            label_name VARCHAR(255) NOT NULL,
            label_dir VARCHAR(1000) NOT NULL,
            label_format VARCHAR(10) NOT NULL,
            label VARCHAR(20) NOT NULL,
            box_xtl INT,
            box_ytl INT,
            box_xbr INT,
            box_ybr INT);''')

In [4]:
query2 = (f'''CREATE TABLE IF NOT EXISTS zer01ne_landmark_info(
            id INT PRIMARY KEY AUTO_INCREMENT,
            img_name VARCHAR(255) NOT NULL,
            img_dir VARCHAR(1000) NOT NULL,
            img_format VARCHAR(10) NOT NULL,
            img_width INT NOT NULL,
            img_height INT NOT NULL,
            color_space VARCHAR(10) NOT NULL,
            label_name VARCHAR(255) NOT NULL,
            label_dir VARCHAR(1000) NOT NULL,
            label_format VARCHAR(10) NOT NULL,
            label VARCHAR(20) NOT NULL,
            landmark VARCHAR(1000));''')

In [3]:
query1_1 = (f'''INSERT INTO zer01ne_box_info 
        (img_name, img_dir, img_format, img_width, img_height, color_space,
        label_name, label_dir, label_format,
        label, box_xtl, box_ytl, box_xbr, box_ybr)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')
query2_1 = (f'''INSERT INTO zer01ne_landmark_info 
        (img_name, img_dir, img_format, img_width, img_height, color_space,
        label_name, label_dir, label_format,
        label, landmark)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')

In [6]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [7]:
try:
    doUT.create_table(query2)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [4]:
labels = {0:'face', 1:'body', 2:'hand', 3:'glasses', 4:'sun-glass', 
          5:'cell-phone', 6:'cigar', 7:'bottle',8:'can', 9:'mask', 10:'wheel'}

In [12]:
value_list1 = []
value_list2 = []
error_list = []

label_path = 'Z:\\zer01ne\\data\\IR\\labeldata'
img_path = 'Z:\\zer01ne\\data\\IR\\rawdata'
label_lists = os.listdir(label_path)

for label_list in label_lists:
    label_files = os.listdir(os.path.join(label_path,label_list))
    total_cnt = len(label_files)
    for cnt,label_file in enumerate(label_files):
        label_dir = os.path.join(label_path,label_list,label_file)
        label_name, label_format = os.path.splitext(os.path.basename(label_file))
        label_format = label_format.replace('.','')
        color_space = 'IR'
        if label_list == 'driver_landmark':
            print(label_file)
            if label_file == 'face_annotations.xml':
                tree = ET.parse(label_dir)
                for cnt,i in enumerate(tree.findall('.//image')):
                    img_file = i.attrib['name']
                    img_width, img_height = i.attrib['width'], i.attrib['height']
                    skeletons = i.findall('skeleton')
                    
                    label = 'face_68'
                    landmarks = []
                    for skeleton in skeletons:
                        
                        for point in skeleton:
                            x,y = point.attrib['points'].split(',')
                            points = (int(float(x)),int(float(y)))
                            landmarks.append(points)
                    # print(len(landmark))        
                    landmark = ",".join(map(str,[(x,y) for x,y in landmarks]))    
                   

                    img_dir = f'{img_path}\\driver_data_img\\{img_file}'

                    img_name, img_format = os.path.splitext(img_file)
                    img_format = img_format.replace('.','')
                    image = cv2.imread(img_dir)
                    img_width = image.shape[1]
                    img_height = image.shape[0]
                    img_dir = img_dir.replace('Z:','DataBase')
                    label_dir = label_dir.replace('Z:','DataBase')
                    value1 = (img_name, img_dir, img_format, img_width, img_height, color_space,
                    label_name, label_dir, label_format, label, landmark)
                    value_list1.append(value1)
                    if len(value_list1) >= 500:
                        print(value_list1[0])
                        print(f'landmark {cnt}')
                        doUT.insert_dataset_values(query2_1, value_list1)
                        value_list1 = []
            elif label_file == 'body_annotations.xml':
                pass
            else:
            # print(label_dir)
                tree = ET.parse(label_dir)
                for cnt,i in enumerate(tree.findall('.//image')):
                    img_file = i.attrib['name']
                    img_width, img_height = i.attrib['width'], i.attrib['height']
                    skeletons = i.findall('skeleton')
                    try:
                        for skeleton in skeletons:
                            label = skeleton.attrib['label']
                            landmarks = []
                            for point in skeleton:
                                x,y = point.attrib['points'].split(',')
                                points = (int(float(x)),int(float(y)))
                                landmarks.append(points)
                                landmark = ",".join(map(str,[(x,y) for x,y in landmarks]))

                            img_dir = f'{img_path}\\driver_data_img\\{img_file}'

                            img_name, img_format = os.path.splitext(img_file)
                            img_format = img_format.replace('.','')
                            image = cv2.imread(img_dir)
                            img_width = image.shape[1]
                            img_height = image.shape[0]
                            img_dir = img_dir.replace('Z:','DataBase')
                            label_dir = label_dir.replace('Z:','DataBase')
                            value1 = (img_name, img_dir, img_format, img_width, img_height, color_space,
                            label_name, label_dir, label_format, label, landmark)
                            value_list1.append(value1)
                        if len(value_list1) >= 500:
                            print(value_list1[0])
                            print(f'landmark {cnt}')
                            doUT.insert_dataset_values(query2_1, value_list1)
                            value_list1 = []
                    except:
                        pass
                        
        else:
            pass
#             img_file = os.path.basename(label_file).replace('.txt','.jpg')
#             img_dir = f'{img_path}\\driver_data_img\\{img_file}'
#             img_name, img_format = os.path.splitext(img_file)
#             img_format = img_format.replace('.','')
#             # print(img_dir)
#             img = cv2.imread(img_dir)
            
            
#             img_width = img.shape[0]
#             img_height = img.shape[1]
            
            
#             with open(label_dir,'r') as f:
#                 data = f.readlines()

#             for i in data:
#                 font = cv2.FONT_HERSHEY_SIMPLEX
#                 font_scale = 0.5
#                 color = (255, 255, 255)  # White color
#                 thickness = 1
                
#                 # print(i)
#                 label_num, x_c, y_c, box_height, box_width = map(float,(i.split(' ')))
#                 label = labels[int(label_num)]
#                 x_c, y_c, box_height, box_width = int(x_c * img_width) , int(y_c * img_height), int(box_height * img_height), int(box_width * img_width)
#                 box_xtl, box_ytl, box_xbr, box_ybr = int(x_c-box_width/2), int(y_c-box_height/2), int(y_c+box_height/2), int(x_c+box_width/2)
#                 org = (box_xtl, box_ytl - 10)
#                 image = cv2.rectangle(img, (box_xtl, box_ytl), (box_xbr, box_ybr), (0, 255, 0), 2)
#                 image = cv2.circle(img, (x_c, y_c), 5, (0, 255, 0), -1)
#                 image = cv2.putText(img, label, org, font, font_scale, color, thickness, cv2.LINE_AA)
#                 cv2.imshow('Bounding Box with Label', img)
#                 cv2.waitKey(0)
#                 cv2.destroyAllWindows()
#                 img_dir = img_dir.replace('Z:','DataBase')
#                 label_dir = label_dir.replace('Z:','DataBase')
#                 value2 = (img_name, img_dir, img_format, img_width, img_height, color_space,
#                          label_name, label_dir, label_format,
#                          label, box_xtl, box_ytl, box_xbr, box_ybr)
#                 value_list2.append(value2)
#                 if len(value_list2) >= 500:
#                     # print(value_list1[0])
#                     print(f'bbox {cnt} / {total_cnt}')
#                     doUT.insert_dataset_values(query1_1, value_list2)
#                     value_list2 = []
        
doUT.insert_dataset_values(query2_1, value_list1)
# print(value_list2)
value_list1 = []
print('done')      

body_annotations.xml
face_annotations.xml
('week6_DRIVER0004_1748-0001', 'DataBase\\zer01ne\\data\\IR\\rawdata\\driver_data_img\\week6_DRIVER0004_1748-0001.jpg', 'jpg', 1920, 1080, 'IR', 'face_annotations', 'DataBase\\zer01ne\\data\\IR\\labeldata\\driver_landmark\\face_annotations.xml', 'xml', 'face_68', '(879, 446),(872, 473),(867, 509),(876, 534),(893, 559),(916, 574),(942, 587),(967, 599),(989, 611),(1009, 621),(1041, 625),(1071, 611),(1087, 593),(1098, 566),(1107, 545),(1115, 527),(1118, 509),(1074, 442),(1084, 440),(1098, 444),(1105, 457),(1095, 455),(1082, 451),(1084, 391),(1099, 393),(1109, 398),(1116, 405),(1120, 413),(973, 357),(988, 352),(1006, 352),(1024, 356),(1041, 367),(976, 405),(993, 400),(1006, 404),(1015, 415),(1002, 415),(987, 412),(1052, 427),(1054, 441),(1056, 455),(1058, 470),(1014, 481),(1028, 480),(1041, 487),(1055, 490),(1061, 497),(978, 520),(996, 514),(1016, 509),(1034, 514),(1052, 520),(1061, 535),(1058, 563),(1048, 557),(1036, 549),(1024, 541),(1011, 539),(

NameError: name 'query2' is not defined